In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import pickle

%matplotlib inline
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
path = '/content/gdrive/Shareddrives/STATS 170A Final Project/datasets/'

In [5]:
def load_data(fname):
    df = pd.DataFrame()
    with open(path+'Data_Frames/'+fname,'rb') as infile:
        df = pickle.load(infile)
    return df

In [6]:
def save_data(df,fname):
    with open(path+'Data_Frames/'+fname, 'wb') as outfile:
        pickle.dump(df,outfile)
    return 0

In [7]:
df_tx_hur = pd.DataFrame()
with open(path+'Data_Frames/df_tx_hur.pkl','rb') as infile:
    df_tx_hur = pickle.load(infile)

In [8]:
df_tx_hur

,fips_county_code,week_end,product_group_code,sale,fips_state_code,hurricane
0,1,20170121,0501,2136.56,48,0.0
1,1,20170121,0503,8792.97,48,0.0
2,1,20170121,0504,615.75,48,0.0
3,1,20170121,0505,865.40,48,0.0
4,1,20170121,0506,1824.40,48,0.0
...,...,...,...,...,...,...
412995,499,20171230,6015,1575.13,48,0.0
412996,499,20171230,6016,1118.12,48,0.0
412997,499,20171230,6017,865.85,48,0.0
412998,499,20171230,6018,6255.04,48,0.0


In [9]:
df_tx_2017 = pd.DataFrame()
with open(path+'Data_Frames/df_tx_2017.pkl','rb') as infile:
    df_tx_2017 = pickle.load(infile)

In [10]:
df_tx_2017

,fips_state_code,fips_county_code,week_end,product_code,product_group_code,product_module_code,sale,last_week_sale,last_week_sale_diff,units,last_week_units,last_week_units_diff,num_stores
2,48,1,20170121,0501_1272,0501,1272,1627.41,1692.96,-1447.54,91,104.0,-87.0,2
3,48,1,20170128,0501_1272,0501,1272,1338.32,1627.41,-65.55,79,91.0,-13.0,2
4,48,1,20170204,0501_1272,0501,1272,1361.99,1338.32,-289.09,76,79.0,-12.0,2
5,48,1,20170211,0501_1272,0501,1272,4425.84,1361.99,23.67,266,76.0,-3.0,2
6,48,1,20170218,0501_1272,0501,1272,1639.04,4425.84,3063.85,96,266.0,190.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,48,507,20171202,9599_6073,9599,6073,10.00,0.00,0.00,2,0.0,0.0,1
48,48,507,20171209,9599_6073,9599,6073,0.00,10.00,10.00,0,2.0,2.0,0
49,48,507,20171216,9599_6073,9599,6073,30.00,0.00,-10.00,3,0.0,-2.0,1
50,48,507,20171223,9599_6073,9599,6073,40.00,30.00,30.00,4,3.0,3.0,1


In [11]:
df_tx_2017.head()

,fips_state_code,fips_county_code,week_end,product_code,product_group_code,product_module_code,sale,last_week_sale,last_week_sale_diff,units,last_week_units,last_week_units_diff,num_stores
2,48,1,20170121,0501_1272,0501,1272,1627.41,1692.96,-1447.54,91,104.0,-87.0,2
3,48,1,20170128,0501_1272,0501,1272,1338.32,1627.41,-65.55,79,91.0,-13.0,2
4,48,1,20170204,0501_1272,0501,1272,1361.99,1338.32,-289.09,76,79.0,-12.0,2
5,48,1,20170211,0501_1272,0501,1272,4425.84,1361.99,23.67,266,76.0,-3.0,2
6,48,1,20170218,0501_1272,0501,1272,1639.04,4425.84,3063.85,96,266.0,190.0,2


## County Filtering

In [12]:
county_0 = df_tx_2017[df_tx_2017['sale']==0]['fips_county_code'].value_counts()

In [13]:
c_list = []
for i in county_0.keys():
    if county_0[i]/55500 < 0.33:
        c_list.append(i)

In [14]:
len(c_list)

70

In [15]:
filter_county = df_tx_2017[df_tx_2017['fips_county_code'].isin(c_list)]

In [16]:
group_agg = filter_county.groupby(['fips_county_code','week_end','product_group_code']).agg({'sale':np.sum})

In [17]:
group_agg = group_agg.reset_index()

In [18]:
np.sum(group_agg['sale']==0)/413000

0.024317191283292978

In [19]:
group_agg['fips_state_code'] = 48

In [20]:
df_hm = pd.read_csv(path+'helpful/hurricane_marker.csv')
df_hm_tx = df_hm[df_hm['fips_state_code']==48]
df_hm_tx = df_hm_tx[df_hm_tx['week_end']>20170000]
df_hm_tx = df_hm_tx[df_hm_tx['week_end']<20180000]

In [21]:
df_tx_hur = pd.merge(group_agg, df_hm_tx[['fips_state_code','fips_county_code','week_end','hurricane']], on=['fips_state_code','fips_county_code','week_end'])

In [22]:
df_tx_hur.columns

Index(['fips_county_code', 'week_end', 'product_group_code', 'sale',
       'fips_state_code', 'hurricane'],
      dtype='object')

In [23]:
df_tx_hur.hurricane.value_counts()

0.0    408752
1.0      4248
Name: hurricane, dtype: int64

In [24]:
df_tx_hur

,fips_county_code,week_end,product_group_code,sale,fips_state_code,hurricane
0,1,20170121,0501,2136.56,48,0.0
1,1,20170121,0503,8792.97,48,0.0
2,1,20170121,0504,615.75,48,0.0
3,1,20170121,0505,865.40,48,0.0
4,1,20170121,0506,1824.40,48,0.0
...,...,...,...,...,...,...
412995,499,20171230,6015,1575.13,48,0.0
412996,499,20171230,6016,1118.12,48,0.0
412997,499,20171230,6017,865.85,48,0.0
412998,499,20171230,6018,6255.04,48,0.0


In [25]:
with open(path+'Data_Frames/df_tx_hur.pkl','wb') as outfile:
    pickle.dump(df_tx_hur, outfile)

## Random Forest

In [26]:
df_tx_hur = df_tx_hur[['sale', 'fips_state_code', 'fips_county_code', 'week_end',
       'product_group_code', 'hurricane']]

In [27]:
df_tx_hur = df_tx_hur.sort_values(['fips_county_code','product_group_code','week_end']).reset_index()

In [28]:
df_tx_hur.columns

Index(['index', 'sale', 'fips_state_code', 'fips_county_code', 'week_end',
       'product_group_code', 'hurricane'],
      dtype='object')

calculate last_week_sale and last_week_sale_diff

In [29]:
last_county = 1
last_group = '0501'
last_sale = -1
last_week_sale = []
#last_week_sale_diff = []
for i in range(len(df_tx_hur)):
    if df_tx_hur['fips_county_code'][i] == last_county and df_tx_hur['product_group_code'][i] == last_group:
        if last_sale != -1:
            last_week_sale.append(last_sale)
            last_sale = df_tx_hur['sale'][i]
        else:
            last_week_sale.append(-1)
            last_sale = df_tx_hur['sale'][i]
    else:
        last_week_sale.append(-1)
        last_county = df_tx_hur['fips_county_code'][i]
        last_group = df_tx_hur['product_group_code'][i]
        last_sale = df_tx_hur['sale'][i]


In [30]:
last_week_sale = np.array(last_week_sale)

In [31]:
last_week_sale_diff = []
last = -1
for i in last_week_sale:
    if i == -1:
        last_week_sale_diff.append(-1)
        last = i
    else:
        if last == -1:
            last_week_sale_diff.append(-1)
            last = i
        else:
            last_week_sale_diff.append(i-last)
            last = i

In [32]:
len(last_week_sale_diff)

413000

In [33]:
df_tx_hur['last_week_sale'] = last_week_sale
df_tx_hur['last_week_sale_diff'] = np.array(last_week_sale_diff)

In [34]:
df_tx_hur['last_week_sale'].replace({-1:np.nan}, inplace=True)
df_tx_hur['last_week_sale_diff'].replace({-1:np.nan}, inplace=True)
df_tx_data = df_tx_hur.dropna()

In [35]:
df_tx_hur = df_tx_hur.sort_values('week_end')

In [36]:
df_tx_hur['week_end'].unique()[:45]

array([20170121, 20170128, 20170204, 20170211, 20170218, 20170225,
       20170304, 20170311, 20170318, 20170325, 20170401, 20170408,
       20170415, 20170422, 20170429, 20170506, 20170513, 20170520,
       20170527, 20170603, 20170610, 20170617, 20170624, 20170701,
       20170708, 20170715, 20170722, 20170729, 20170805, 20170812,
       20170819, 20170826, 20170902, 20170909, 20170916, 20170923,
       20170930, 20171007, 20171014, 20171021, 20171028, 20171104,
       20171111, 20171118, 20171125])

In [37]:
df_tx_data2 = load_data('df_tx_data2.pkl')

In [38]:
df_tx_data.head()

,index,sale,fips_state_code,fips_county_code,week_end,product_group_code,hurricane,last_week_sale,last_week_sale_diff
2,236,1893.54,48,1,20170204,0501,0.0,1783.48,-353.08
3,354,5080.40,48,1,20170211,0501,0.0,1893.54,110.06
4,472,2068.19,48,1,20170218,0501,0.0,5080.40,3186.86
5,590,1766.81,48,1,20170225,0501,0.0,2068.19,-3012.21
6,708,1645.77,48,1,20170304,0501,0.0,1766.81,-301.38


In [41]:
feat_col = ['week_end','fips_county_code','product_group_code', 'hurricane', 'last_week_sale', 'last_week_sale_diff']

In [42]:
x_train = df_tx_data[df_tx_data['week_end']<=20171125][feat_col]
y_train = df_tx_data[df_tx_data['week_end']<=20171125]['sale']

x_test = df_tx_data[df_tx_data['week_end']>20171125][feat_col]
y_test = df_tx_data[df_tx_data['week_end']>20171125]['sale']

In [43]:
model = RandomForestRegressor(n_estimators=10, random_state=0)
model.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [44]:
ypred = model.predict(x_train)
np.mean(np.abs(ypred-y_train))

599.4931443379274

In [45]:
ypred_test = model.predict(x_test)
np.mean(np.abs(ypred_test-y_test))

4611.033314316145

In [46]:
x_train

,week_end,fips_county_code,product_group_code,hurricane,last_week_sale,last_week_sale_diff
2,20170204,1,0501,0.0,1783.48,-353.08
3,20170211,1,0501,0.0,1893.54,110.06
4,20170218,1,0501,0.0,5080.40,3186.86
5,20170225,1,0501,0.0,2068.19,-3012.21
6,20170304,1,0501,0.0,1766.81,-301.38
...,...,...,...,...,...,...
412990,20171028,499,9599,0.0,0.00,0.00
412991,20171104,499,9599,0.0,0.00,0.00
412992,20171111,499,9599,0.0,0.00,0.00
412993,20171118,499,9599,0.0,0.00,0.00


In [47]:
feat_col

['week_end',
 'fips_county_code',
 'product_group_code',
 'hurricane',
 'last_week_sale',
 'last_week_sale_diff']

In [48]:
pd.Series(model.feature_importances_, index=feat_col)

week_end               0.002266
fips_county_code       0.001467
product_group_code     0.007147
hurricane              0.000705
last_week_sale         0.980249
last_week_sale_diff    0.008166
dtype: float64

In [49]:
model.feature_importances_

array([2.26583516e-03, 1.46743015e-03, 7.14697590e-03, 7.04678635e-04,
       9.80248696e-01, 8.16638445e-03])

In [51]:
df_tx_data[df_tx_data['week_end']<=20171125]

,index,sale,fips_state_code,fips_county_code,week_end,product_group_code,hurricane,last_week_sale,last_week_sale_diff
2,236,1893.54,48,1,20170204,0501,0.0,1783.48,-353.08
3,354,5080.40,48,1,20170211,0501,0.0,1893.54,110.06
4,472,2068.19,48,1,20170218,0501,0.0,5080.40,3186.86
5,590,1766.81,48,1,20170225,0501,0.0,2068.19,-3012.21
6,708,1645.77,48,1,20170304,0501,0.0,1766.81,-301.38
...,...,...,...,...,...,...,...,...,...
412990,411937,0.00,48,499,20171028,9599,0.0,0.00,0.00
412991,412055,0.00,48,499,20171104,9599,0.0,0.00,0.00
412992,412173,0.00,48,499,20171111,9599,0.0,0.00,0.00
412993,412291,0.00,48,499,20171118,9599,0.0,0.00,0.00


In [54]:
x_train = df_tx_data[df_tx_data['week_end']<=20171125][feat_col]
y_train = df_tx_data[df_tx_data['week_end']<=20171125]['sale']

x_test = df_tx_data[df_tx_data['week_end']>20171125][feat_col]
y_test = df_tx_data[df_tx_data['week_end']>20171125]['sale']

In [55]:
model2 = RandomForestRegressor(n_estimators=10, random_state=0)
model2.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [56]:
ypred = model2.predict(x_train)
np.mean(np.abs(ypred-y_train))

599.4931443379274

In [57]:
ypred_test = model2.predict(x_test)
np.mean(np.abs(ypred_test-y_test))

4611.033314316145

In [61]:
!pip3 install shap

     |████████████████████████████████| 358kB 13.1MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491605 sha256=9b7dd4b8de4e19d541943738b3f114638b312103b6461e27b524659afd6ed93e
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_train)

In [ ]:
shap.summary_plot(shap_values, features=x_train, feature_names=x_train.columns)

## CatBoost

In [ ]:
!pip install catboost

In [ ]:
df_tx_data = load_data('df_tx_data.pkl')

In [ ]:
df_fema_county = load_data('df_fema_county.pkl')

In [ ]:
df_fema_county.head()

In [ ]:
df_fema_county = df_fema_county[['STATEFIPS','COUNTYFIPS','SOVI_SCORE','HRCN_EVNTS','HRCN_AFREQ','HRCN_EXPP','HRCN_RISKS']].rename(columns={'STATEFIPS':'fips_state_code',
'COUNTYFIPS':'fips_county_code'})

In [ ]:
df_tx_data2 = pd.merge(df_tx_data, df_fema_county, on=['fips_state_code','fips_county_code'])

In [ ]:
save_data(df_tx_data2, 'df_tx_data2.pkl')

In [ ]:
df_tx_data3 = load_data('df_tx_data3.pkl')

In [ ]:
feat_col = ['week_end_date','fips_county_code','product_group_code', 'hurricane', 'last_week_sale', 'last_week_sale_diff','SOVI_SCORE','HRCN_EVNTS','HRCN_AFREQ','HRCN_EXPP','HRCN_RISKS']

In [ ]:
x_train = df_tx_data2[df_tx_data2['week_end']<=20171125][feat_col]
y_train = df_tx_data2[df_tx_data2['week_end']<=20171125]['sale']

x_test = df_tx_data2[df_tx_data2['week_end']>20171125][feat_col]
y_test = df_tx_data2[df_tx_data2['week_end']>20171125]['sale']

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
#model_cb = CatBoostRegressor(random_state=0, task_type='GPU',learning_rate=1, iterations=10000)
model_cb = CatBoostRegressor(random_state=0, iterations=1000)

In [ ]:
model_cb.fit(x_train,y_train,[1,2])

In [ ]:
np.mean(np.abs(y_train-model_cb.predict(x_train)))

In [ ]:
np.mean(np.abs(y_test-model_cb.predict(x_test)))

In [ ]:
np.mean(np.exp(y_test-model_cb.predict(x_test))**2)

In [ ]:
x_train.head()

In [ ]:
model_cb.get_feature_importance()

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model_cb)
shap_values = explainer.shap_values(x_train)

In [ ]:
shap.summary_plot(shap_values, features=x_train, feature_names=x_train.columns)

## linear mixed effect

In [ ]:
df_tx_data = df_tx_data.reset_index().drop(columns=['level_0','index'])

In [ ]:
df_tx_data.head()

In [ ]:
df_tx_data['week_end_date'] = df_tx_data['week_end']-20170000

In [ ]:
df_tx_data['log_sale'] = np.log(df_tx_data['sale']).replace({-np.inf:0})

In [ ]:
sns.distplot(df_tx_data['sale'])
plt.show()

In [ ]:
df_tx_data = load_data('df_tx_data.pkl')

In [ ]:
df_fema_county = load_data('df_fema_county.pkl')

In [ ]:
df_fema_county.head()

In [ ]:
df_tx_data.head()

### Models

In [ ]:
df_tx_data2 = load_data('df_tx_data2.pkl')

In [ ]:
np.mean(df_tx_data2['sale'])

In [ ]:
df_tx_data2.head()

In [ ]:
md1 = smf.mixedlm("sale ~ last_week_sale+last_week_sale_diff+hurricane", df_tx_data2, groups=df_tx_data2['fips_county_code'])
mdf1 = md1.fit()

In [ ]:
print(mdf1.summary())

In [ ]:
ypred = mdf1.predict()
y_true = df_tx_data2['sale']
np.mean(np.abs(y_true-ypred))

In [ ]:
md2 = smf.mixedlm("sale ~ last_week_sale+last_week_sale_diff+hurricane", df_tx_data2, groups=df_tx_data2['product_group_code'])
mdf2 = md2.fit()

In [ ]:
print(mdf2.summary())

In [ ]:
ypred = mdf2.predict()
y_true = df_tx_data2['sale']
np.mean(np.abs(y_true-ypred))

In [ ]:
md3 = smf.mixedlm("sale ~ last_week_sale+last_week_sale_diff+product_group_code+hurricane", df_tx_data2, groups=df_tx_data2['fips_county_code'])
mdf3 = md3.fit()

In [ ]:
print(mdf3.summary())

In [ ]:
mdf3.fe_params

In [ ]:
ypred = mdf3.predict()
y_true = df_tx_data2['sale']
np.mean(np.abs(y_true-ypred))

---

In [ ]:
md2 = smf.mixedlm("sale ~ week_end+product_group_code+hurricane+last_week_sale+last_week_sale_diff", df_tx_data, groups=df_tx_data["fips_county_code"])
mdf2 = md2.fit()

In [ ]:
print(mdf2.summary())

## Past Attempts

In [ ]:
feat_col = ['fips_county_code', 'product_code',
       'product_group_code', 'product_module_code', 'last_week_sale',
       'last_week_sale_diff', 'units', 'last_week_units',
       'last_week_units_diff', 'num_stores', 'hurricane']

In [ ]:
x_train = df_tx_hur[df_tx_hur['week_end']<=20171125][feat_col]
y_train = df_tx_hur[df_tx_hur['week_end']<=20171125]['sale']

x_test = df_tx_hur[df_tx_hur['week_end']>20171125][feat_col]
y_test = df_tx_hur[df_tx_hur['week_end']>20171125]['sale']

In [ ]:
with open(path+'Data_Frames/x_train.pkl','wb') as outfile:
  pickle.dump(x_train, outfile)
with open(path+'Data_Frames/y_train.pkl','wb') as outfile:
  pickle.dump(y_train, outfile)
with open(path+'Data_Frames/x_test.pkl','wb') as outfile:
  pickle.dump(x_test, outfile)
with open(path+'Data_Frames/y_test.pkl','wb') as outfile:
  pickle.dump(y_test, outfile)

In [ ]:
x_train = pd.DataFrame()
y_train = pd.DataFrame()
x_test = pd.DataFrame()
y_test = pd.DataFrame()
with open(path+'Data_Frames/x_train.pkl','rb') as infile:
  x_train = pickle.load(infile)
with open(path+'Data_Frames/y_train.pkl','rb') as infile:
  y_train = pickle.load(infile)
with open(path+'Data_Frames/x_test.pkl','rb') as infile:
  x_test = pickle.load(infile)
with open(path+'Data_Frames/y_test.pkl','rb') as infile:
  y_test = pickle.load(infile)

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_estimators=10, random_state=0)
model.fit(x_train,y_train)

In [ ]:
with open(path+'Data_Frames/rf_1.pkl','wb') as outfile:
    pickle.dump(model, outfile)

In [ ]:
model = RandomForestRegressor(n_estimators=10, random_state=0)
with open(path+'Data_Frames/rf_1.pkl','rb') as infile:
    model = pickle.load(infile)

In [ ]:
ypred = model.predict(x_train)

In [ ]:
np.mean((ypred-y_train)**2)

In [ ]:
ypred[:150]

In [ ]:
y_train[:150]